<a href="https://colab.research.google.com/github/faragodian/Calculadora-Respuesta-Tumoral/blob/main/OP_Problema_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problema 1:
**Automatización del cálculo de la respuesta en enfermedades tumorales**

En enfermedades tumorales, como el mieloma, se mide el *componente monoclonal sérico* al diagnóstico (a) y posterior al tratamiento (b). La respuesta se calcula mediante la fórmula

\[
(1 - a/b) * 100
\]

y se clasifica en:

**Completa:** si desaparece el componente.

**Muy buena respuesta parcial:** disminución >90 %.

**Respuesta parcial:** disminución >50 % y <90 %.

**Sin respuesta:** disminución <50 %.

En linfomas, el cociente es similar, pero basado en el múltiplo de los diámetros de las lesiones al diagnóstico y después del tratamiento.

In [3]:
"""
Clasificacion según el enunciado Problema 1 OP:
  - Completa: si desaparece el componente (b=0)
  - MBPR: >90% (o ≥90% si inclusivo=True)
  - RP: >50% y <90% (o ≥50% y <90% si inclusivo=True)
  - Sin respuesta: <50%
   OJO! Añadimos 'Progresión' si respuesta_pct < 0.
"""
from dataclasses import dataclass
from typing import Optional, Literal, List, Dict

Disease = Literal["mieloma", "linfoma", "otro"]

@dataclass
class ResponseResult:
    enfermedad: Disease
    a: float              # valor basal (diagnóstico)
    b: float              # valor post-tratamiento
    respuesta_pct: float  # porcentaje de disminución (positivo = mejora)
    categoria: str
    nota: Optional[str] = None

def calcular_respuesta_porcentaje(a: float, b: float) -> float:
    if a < 0 or b < 0:
        raise ValueError("Los valores a y b deben ser no negativos.")
    if b == 0:
        return 100.0
    if a == 0:
        return 100.0
    return (1 - (a / b)) * 100.0

def clasificar_respuesta(respuesta_pct: float, desaparece: bool,
                         inclusivo: bool = True) -> str:
    if desaparece:
        return "Completa"
    if respuesta_pct < 0:
        return "Progresión"
    if inclusivo:
        if respuesta_pct >= 90:
            return "Muy buena respuesta parcial"
        if 50 <= respuesta_pct < 90:
            return "Respuesta parcial"
        return "Sin respuesta"
    else:
        if respuesta_pct > 90:
            return "Muy buena respuesta parcial"
        if 50 < respuesta_pct < 90:
            return "Respuesta parcial"
        return "Sin respuesta"

def evaluar_respuesta(enfermedad: Disease, a: float, b: float,
                      inclusivo: bool = True) -> ResponseResult:
    nota = None
    if a == 0:
        nota = "a=0 (valor basal nulo); porcentaje interpretado como 100%."
    respuesta = calcular_respuesta_porcentaje(a, b)
    categoria = clasificar_respuesta(respuesta, desaparece=(b == 0), inclusivo=inclusivo)
    return ResponseResult(
        enfermedad=enfermedad,
        a=a,
        b=b,
        respuesta_pct=round(respuesta, 2),
        categoria=categoria,
        nota=nota
    )

"""
Misma ratio a/b para mieloma (componente monoclonal) y linfoma (producto de diámetros).
"""

print("=== Evaluador de respuesta tumoral ===")
enfermedad = input("Tipo de enfermedad (mieloma / linfoma / otro): ").strip().lower()
a = float(input("Valor basal (diagnóstico): "))
b = float(input("Valor post-tratamiento: "))

resultado = evaluar_respuesta(enfermedad, a, b)

print("\n--- RESULTADO ---")
print(f"Enfermedad: {resultado.enfermedad}")
print(f"Valor basal (a): {resultado.a}")
print(f"Valor post-tratamiento (b): {resultado.b}")
print(f"Disminución porcentual: {resultado.respuesta_pct}%")
print(f"Clasificación: {resultado.categoria}")
if resultado.nota:
    print(f"Nota: {resultado.nota}")


=== Evaluador de respuesta tumoral ===
Tipo de enfermedad (mieloma / linfoma / otro): mieloma
Valor basal (diagnóstico): 5
Valor post-tratamiento: 4

--- RESULTADO ---
Enfermedad: mieloma
Valor basal (a): 5.0
Valor post-tratamiento (b): 4.0
Disminución porcentual: -25.0%
Clasificación: Progresión
